In [2]:
import os
# These are the default AWS configurations for the Analysis Sandbox.
# that are set in the environmnet variables. 
aws_default_config = {
    #'AWS_NO_SIGN_REQUEST': 'YES', 
    'AWS_SECRET_ACCESS_KEY': 'fake',
    'AWS_ACCESS_KEY_ID': 'fake',
}

# To access public bucket, need to remove the AWS credentials in 
# the environment variables or the following error will occur.
# PermissionError: The AWS Access Key Id you provided does not exist in our records.

for key in aws_default_config.keys():
    if key in os.environ:
        del os.environ[key]

In [3]:
import json
import logging
import os
import sqlite3
from importlib import import_module

import click
import datacube
import fsspec
from deafrica_conflux.cli.logs import logging_setup
from deafrica_conflux.drill import drill
from deafrica_conflux.io import (
    check_dir_exists,
    check_file_exists,
    check_if_s3_uri,
    table_exists,
    write_table_to_parquet,
)
from deafrica_conflux.plugins.utils import run_plugin, validate_plugin
from odc import dscache
from odc.aws import s3_download
from rasterio.errors import RasterioIOError
import deafrica_conflux.db

In [4]:
verbose = 1
# File path to the cache file database.
cachedb_file_path = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/dbs/wofs_ls_2023-03--P3M.db"
# Text file to get tasks ids from.
tasks_text_file = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/tasks/wofs_ls_2023-03--P3M_tasks.txt"
# Name of the plugin. Plugin file must be in the
# deafrica_conflux/plugins/ directory.
plugin_name = "waterbodies_timeseries"
# Path to the directory containing the polygons raster files.
polygons_rasters_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters"
# JSON file containing the polygons ids.
polygon_numericids_to_stringids_file = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/polygon_numericids_to_stringids.json"
# Directory to write the drill outputs to.
output_directory = "s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/drill_output_parquet_files"
# Rerun tasks that have already been processed.
overwrite = True
# Write drill outputs to db
db = True

In [5]:
# Set up logger.
logging_setup(verbose)
_log = logging.getLogger(__name__)

In [6]:
# Support pathlib Paths
cachedb_file_path = str(cachedb_file_path)
tasks_text_file = str(tasks_text_file)
polygons_rasters_directory = str(polygons_rasters_directory)
output_directory = str(output_directory)
if polygon_numericids_to_stringids_file:
    polygon_numericids_to_stringids_file = str(polygon_numericids_to_stringids_file)

In [7]:
# Read the plugin as a Python module.
module = import_module(f"deafrica_conflux.plugins.{plugin_name}")
plugin_file = module.__file__
plugin = run_plugin(plugin_file)
_log.info(f"Using plugin {plugin_file}")
validate_plugin(plugin)

# Get the drill name from the plugin
drill_name = plugin.product_name

[2024-02-08 20:35:30,925] {2585789683.py:5} INFO - Using plugin /home/jovyan/dev/deafrica-conflux/deafrica_conflux/plugins/waterbodies_timeseries.py


In [8]:
if polygon_numericids_to_stringids_file:
    if not check_file_exists(polygon_numericids_to_stringids_file):
        _log.error(f"File {polygon_numericids_to_stringids_file} does not exist!")
        raise FileNotFoundError(f"File {polygon_numericids_to_stringids_file} does not exist!)")

if not check_dir_exists(polygons_rasters_directory):
    _log.error(f"Directory {polygons_rasters_directory} does not exist!")
    raise FileNotFoundError(f"Directory {polygons_rasters_directory} does not exist!)")

# Create the output directory if it does not exist.
if not check_dir_exists(output_directory):
    if check_if_s3_uri(output_directory):
        fsspec.filesystem("s3").makedirs(output_directory, exist_ok=True)
    else:
        fsspec.filesystem("file").makedirs(output_directory, exist_ok=True)
    _log.info(f"Created directory {output_directory}")

if not check_file_exists(cachedb_file_path):
    _log.error(f"Could not find the database file {cachedb_file_path}!")
    raise FileNotFoundError(f"{cachedb_file_path} does not exist!")
else:
    if check_if_s3_uri(cachedb_file_path):
        cachedb_file_path = s3_download(cachedb_file_path)
        if not check_file_exists(cachedb_file_path):
            _log.error(f"{cachedb_file_path} did not download!")
            raise FileNotFoundError(
                f"{cachedb_file_path} does not exist! File did not download."
            )

if not check_file_exists(tasks_text_file):
    _log.error(f"Could not find the text file {tasks_text_file}!")
    raise FileNotFoundError(f"Could not find text file {tasks_text_file}!")

[2024-02-08 20:35:32,036] {credentials.py:611} INFO - Found credentials in shared credentials file: ~/.aws/credentials


In [9]:
# Read task ids from the S3 URI or File URI.
if check_if_s3_uri(tasks_text_file):
    fs = fsspec.filesystem("s3")
else:
    fs = fsspec.filesystem("file")

with fs.open(tasks_text_file, "r") as file:
    tasks = [line.strip() for line in file]
_log.info(f"Read {len(tasks)} tasks from file.")
_log.debug(f"Read {tasks} from file.")

[2024-02-08 20:35:33,869] {3293637472.py:9} INFO - Read 1715 tasks from file.


In [10]:
# Read the polygons ids mapping file.
if polygon_numericids_to_stringids_file:
    if check_if_s3_uri(polygon_numericids_to_stringids_file):
        fs = fsspec.filesystem("s3")
    else:
        fs = fsspec.filesystem("file")

    with fs.open(polygon_numericids_to_stringids_file) as f:
        polygon_numericids_to_stringids = json.load(f)
else:
    polygon_numericids_to_stringids = {}

In [11]:
# Connect to the datacube
dc = datacube.Datacube(app="deafrica-conflux-drill")

# Read the cache file
cache = dscache.open_ro(cachedb_file_path)

/usr/local/lib/python3.10/dist-packages/datacube/utils/documents.py:216: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  ref_resolver = jsonschema.RefResolver.from_schema(


In [13]:
if db:
    engine = deafrica_conflux.db.get_engine_sqlite_file_db("waterbodies_test.db")

In [16]:
%%time
failed_tasks = []
for i, task in enumerate(tasks):
    _log.info(f"Processing task {task} ({i + 1}/{len(tasks)})")

    if not overwrite:
        _log.info(f"Checking existence of {task}")
        exists = table_exists(
            drill_name=drill_name, task_id_string=task, output_directory=output_directory
        )
    if overwrite or not exists:
        try:
            # Perform the polygon drill.
            table = drill(
                plugin=plugin,
                task_id_string=task,
                cache=cache,
                polygons_rasters_directory=polygons_rasters_directory,
                polygon_numericids_to_stringids=polygon_numericids_to_stringids,
                dc=dc,
            )
            pq_file_name = write_table_to_parquet(  # noqa F841
                drill_name=drill_name,
                task_id_string=task,
                table=table,
                output_directory=output_directory,
            )
            if db:
                _log.info(f"Writing {pq_file_name} to DB")
                deafrica_conflux.db.add_waterbody_observations_to_db(
                    paths=[pq_file_name],
                    verbose=True,
                    engine=engine,
                    drop=False,
                )
            break
        except KeyError as keyerr:
            _log.exception(f"Found task {task} has KeyError: {str(keyerr)}")
            failed_tasks = [].append(task)
        except TypeError as typeerr:
            _log.exception(f"Found task {task} has TypeError: {str(typeerr)}")
            failed_tasks.append(task)
        except RasterioIOError as ioerror:
            _log.exception(f"Found task {task} has RasterioIOError: {str(ioerror)}")
            failed_tasks.append(task)
        except ValueError as valueerror:
            _log.exception(f"Found task {task} has ValueError: {str(valueerror)}")
            failed_tasks.append(task)
        else:
            _log.info(f"Task {task} successful")
    else:
        _log.info(f"Drill outputs for {task} already exist, skipping")
    #break
    if failed_tasks:
        # Write the failed dataset ids to a text file.
        parent_folder, file_name = os.path.split(tasks_text_file)
        file, file_extension = os.path.splitext(file_name)
        failed_tasks_text_file = os.path.join(
            parent_folder, file + "_failed_tasks" + file_extension
        )

        with fs.open(failed_tasks_text_file, "a") as file:
            for task in failed_tasks:
                file.write(f"{task}\n")

        _log.info(f"Failed tasks {failed_tasks} written to: {failed_tasks_text_file}.")

[2024-02-08 20:38:30,888] {<timed exec>:3} INFO - Processing task 2023-03-01/171/092 (1/1715)
[2024-02-08 20:38:30,889] {drill.py:80} INFO - Finding polygon raster tile....
[2024-02-08 20:38:30,891] {io.py:445} INFO - Found 1 GeoTIFF files.
[2024-02-08 20:38:30,901] {drill.py:91} INFO - Loaded s3://deafrica-waterbodies-dev/waterbodies/v0.0.2/senegal_basin/conflux/historical_extent_rasters/x171_y092.tif
[2024-02-08 20:38:30,903] {drill.py:117} INFO - Query object to use for loading data {'resampling': 'nearest', 'dask_chunks': {'x': 3200, 'y': 3200}, 'group_by': 'solar_day', 'like': GeoBox(3200, 3200, Affine(30.0, 0.0, -960000.0,
       0.0, -30.0, 1536000.0), PROJCS["WGS 84 / NSIDC EASE-Grid 2.0 Global",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Cylindrical_Equal_

  0%|          | 0/1 [00:00<?, ?it/s]

2024-02-08 20:38:31,737 INFO sqlalchemy.engine.Engine BEGIN (implicit)
[2024-02-08 20:38:31,737] {base.py:1032} INFO - BEGIN (implicit)
2024-02-08 20:38:31,738 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("waterbodies")
[2024-02-08 20:38:31,738] {base.py:1863} INFO - PRAGMA main.table_info("waterbodies")
2024-02-08 20:38:31,739 INFO sqlalchemy.engine.Engine [raw sql] ()
[2024-02-08 20:38:31,739] {base.py:1868} INFO - [raw sql] ()
2024-02-08 20:38:31,741 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("waterbodies")
[2024-02-08 20:38:31,741] {base.py:1863} INFO - PRAGMA temp.table_info("waterbodies")
2024-02-08 20:38:31,741 INFO sqlalchemy.engine.Engine [raw sql] ()
[2024-02-08 20:38:31,741] {base.py:1868} INFO - [raw sql] ()
2024-02-08 20:38:31,743 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("waterbody_observations")
[2024-02-08 20:38:31,743] {base.py:1863} INFO - PRAGMA main.table_info("waterbody_observations")
2024-02-08 20:38:31,744 INFO sqlalchemy.engine.Engin

100%|██████████| 1/1 [00:00<00:00,  6.18it/s]

CPU times: user 853 ms, sys: 94.4 ms, total: 947 ms
Wall time: 1.01 s


In [20]:
table

,px_total,px_wet,area_wet_m2,px_dry,area_dry_m2,px_invalid,area_invalid_m2,date
ef0uwy5ts8,8,NaN,NaN,NaN,NaN,8,7200,2023-03-01
ef11rnfbh0,20,NaN,NaN,NaN,NaN,20,18000,2023-03-01
ef11tnk07h,14,NaN,NaN,NaN,NaN,14,12600,2023-03-01
ef11tt78ve,11,NaN,NaN,NaN,NaN,11,9900,2023-03-01
ef139buy4m,3251,NaN,NaN,NaN,NaN,3251,2925900,2023-03-01
ef143qykhu,8,NaN,NaN,NaN,NaN,8,7200,2023-03-01
ef1475pckn,8,NaN,NaN,NaN,NaN,8,7200,2023-03-01
ef14zxghnj,33,NaN,NaN,NaN,NaN,33,29700,2023-03-01
ef163b6gzv,9,NaN,NaN,NaN,NaN,9,8100,2023-03-01
ef1640yr52,13,NaN,NaN,NaN,NaN,13,11700,2023-03-01


In [20]:
import pandas as pd
from sqlite3 import connect
pd.read_sql("SELECT * from waterbody_observations", con=connect("waterbodies_test.db"))

,obs_id,uid,px_total,px_wet,area_wet_m2,px_dry,area_dry_m2,px_invalid,area_invalid_m2,date
0,2023-03-01/171/092_ef0uwy5ts8,ef0uwy5ts8,8,None,None,None,None,8.0,7200.0,2023-03-01 00:00:00.000000
1,2023-03-01/171/092_ef11rnfbh0,ef11rnfbh0,20,None,None,None,None,20.0,18000.0,2023-03-01 00:00:00.000000
2,2023-03-01/171/092_ef11tnk07h,ef11tnk07h,14,None,None,None,None,14.0,12600.0,2023-03-01 00:00:00.000000
3,2023-03-01/171/092_ef11tt78ve,ef11tt78ve,11,None,None,None,None,11.0,9900.0,2023-03-01 00:00:00.000000
4,2023-03-01/171/092_ef139buy4m,ef139buy4m,3251,None,None,None,None,3251.0,2925900.0,2023-03-01 00:00:00.000000
5,2023-03-01/171/092_ef143qykhu,ef143qykhu,8,None,None,None,None,8.0,7200.0,2023-03-01 00:00:00.000000
6,2023-03-01/171/092_ef1475pckn,ef1475pckn,8,None,None,None,None,8.0,7200.0,2023-03-01 00:00:00.000000
7,2023-03-01/171/092_ef14zxghnj,ef14zxghnj,33,None,None,None,None,33.0,29700.0,2023-03-01 00:00:00.000000
8,2023-03-01/171/092_ef163b6gzv,ef163b6gzv,9,None,None,None,None,9.0,8100.0,2023-03-01 00:00:00.000000
9,2023-03-01/171/092_ef1640yr52,ef1640yr52,13,None,None,None,None,13.0,11700.0,2023-03-01 00:00:00.000000
